# Title: Santa Barbara Thomas Fire Air Quality Index (AQI) Analysis
Author: Carly Caswell

Repository: https://github.com/ccaswell25/eds220-hwk4-task3 

## About

### Purpose
I will create a false color image to show the location of the Thomas fire in 2017 and the impact of air quality on that specific area during the time period of the fire.  

### Highlights

-Geospatial data exploration and wrangling with geopandas and numpy

-Data wrangling and manipulation to calculate moving averages for air quality index during the period of the Thomas Fire

-Creating and customizing a map of the Thomas fire area 

### Dataset Description and References

**Dataset 1**

A simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite. 

Information about Landsat bands from USGS:

[What are the band designations for the Landsat satellites?](https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites)

[Common Landsat Band Combinations](https://www.usgs.gov/media/images/common-landsat-band-combinations)

[How do I use a scale factor with Landsat Level-2 science products?](https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products)


The data was accessed and pre-processed in the Microsoft Planetary Computer to remove data outside land and coarsen the spatial resolution ([Landsat Collection in MPC](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)). Data should be used for visualization purposes only. 


**Dataset 2**

A shapefile of fire perimeters in California during 2017. 
The [complete file can be accessed in the CA state geoportal](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/about).

 
**Dataset 3**

We are using [Air Quality Index (AQI)](https://www.airnow.gov/aqi/aqi-basics/) data from the [US Environmental Protection Agency](https://www.epa.gov) to visualize the impact on the AQI of the 2017 [Thomas Fire](https://en.wikipedia.org/wiki/Thomas_Fire) in Santa Barbara County. 

# Importing 

In [ ]:
#Importing Libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rioxr
import xarray as xr
import pandas as pd 
from shapely.geometry import Polygon 
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from shapely.geometry import Point

In [ ]:
# Importing Data
#Reading in bands data 
bands = os.path.join(os.getcwd(), 'data', 'landsat8-2018-01-26-sb-simplified.nc') 
bands = rioxr.open_rasterio(bands)

#Reading in CA fires data 
ca_fires = gpd.read_file(os.path.join(os.getcwd(), 'data', 'California_Fire_Perimeters_2017', 'California_Fire_Perimeters_2017.shp'))

#Reading in AQI data for 2017 and 2018
aqi_17 = pd.read_csv("https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip")
aqi_18 = pd.read_csv("https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip")

#add this data to the gitignore

# Data Exploration

In [ ]:
#AQI DATA
#Taking an initial look at my 2017 dataframe:
print(aqi_17.head())
#Taking an initial look at my 2018 dataframe:
print(aqi_18.head())
#Additional data exploration to view the columns of the dataframe:
print(aqi_17.columns)
print(aqi_18.columns)


#CALIFORNIA FIRES DATA 
#Checking the columns of the CA fires data
ca_fires.columns
## Making the CA Fires columns lowercase 
ca_fires.columns = ca_fires.columns.str.lower()
ca_fires.head() #checking the columns were updated

#LANDSAT COLLECTION 2 DATA 
print('height:', bands.rio.height)
print('width:', bands.rio.width)
print('resolution:', bands.rio.resolution())
print('dims:', bands.dims)
#I can see that there is an extra band so I am going to remove it
## Removing the band from the bands dataframe:
bands = bands.squeeze()
bands = bands.drop('band')
print(bands.dims, '\n', bands.coords, '\n') #checking the band was removed from our dataframe

# Geographical Context 
We know have a broad set of data for all counties in 2017 and 2018 and a map that is looking at all California fires. Let's update our data to specifically look at the Thomas Fire and Santa Barbara county and make sure our mapping data is in the same CRS so we can project it on the same scale.

In [ ]:
#First I'm checking the CRS
print(ca_fires.crs)
print(swi_bands.rio.crs)
print(new_bands.rio.crs)

#Creating a false color image 
swi_bands = bands[["swir22", "nir08", "red"]]
swi_bands =swi_bands.to_array()
swi_bands.plot.imshow(robust= True)


#I noticed they are different so I'm going to convert the CA crs
ca_fires = ca_fires.to_crs(32611)
## Making the CRS the same
ca_fires = ca_fires.to_crs(32611)
## Getting just the Thomas fire data
ca_fires_new = ca_fires[ca_fires.fire_name == "THOMAS"]


# Data Analysis
- Include subsections as necessary to guide reader through your analysis
- Include checks to see operations worked
- Checks must be short and informative: print specific attributes instead
of running df.head() or printing entire objects.

In [ ]:
# Combining the dataframes
aqi = pd.concat([aqi_17, aqi_18])

# Fixing the index
aqi = aqi.reset_index(drop=True)
aqi

# initial column names: notice caps and spaces (difficult to work with!)
print(aqi.columns, '\n')

# re-assign the column names - .str.lower() makes them lower case
aqi.columns = aqi.columns.str.lower()
print(aqi.columns, '\n')

#  re-assign the column names again - .str.replace(' ','_') replaces the space for _
aqi.columns = aqi.columns.str.replace(' ','_')
print(aqi.columns)

# as a "one liner" you could achieve this column name cleaning like this:
# aqi.columns = aqi.columns.str.lower().str.replace(' ','_')

# Selecting data from 'Santa Barbara' county
aqi_sb = aqi[aqi['county_name'] == 'Santa Barbara'].copy()

# Dropping specified columns
columns_to_drop = ['state_name', 'county_name', 'state_code', 'county_code']
aqi_sb.drop(columns_to_drop, inplace=True, axis = 1)

# Checking data types of the columns
print(aqi_sb.dtypes)

#The data column is stored as an object and it needs to be converted to datetime!

# Updating the date column to a datetime object
aqi_sb['date'] = pd.to_datetime(aqi_sb['date'])

# Setting the index to the date column
aqi_sb.set_index('date', inplace=True)

# Printing the updated DataFrame
aqi_sb

# Checking the index
aqi_sb.index
#the data type is datetime, which looks correct! 

# rolling() is a method for pandas.series that provides rolling window calculations
# the parameter '5D' indicates we want the window to be 5 days
# This is a lazy method (think groupby), we need to specify what we want to calculate over each window
# here we add the aggregator function mean()
# this indicates we want the mean over each window
# and we get a pd.Series as ouput
aqi_sb.aqi.rolling('5D').mean()

# Adding a new column with the 5-day rolling mean:
aqi_sb['five_day_average'] = aqi_sb.aqi.rolling('5D').mean()


# Final Output

In [ ]:
## Plotting the shapefile
fig, ax = plt.subplots()
fig.set_size_inches(size, size*aspect) #why? bc cannot use ax and size aspect together
swi_bands.plot.imshow(ax=ax, robust = True)
ca_fires_new .plot(ax=ax,facecolor='none', edgecolor='red', linewidth=2, alpha=0.5)

## Set plot title
plt.title('Map of the Thomas Fire Perimeter (2017) in California')

## Showing the plot
plt.show()


#Line plot of the AQI 5-day average:
plt.figure(figsize=(12, 6))
plt.plot(aqi_sb.index, aqi_sb['aqi'], label='Daily AQI', color='cornflowerblue')
plt.plot(aqi_sb.index, aqi_sb['five_day_average'], label='5-Day Average', color='salmon')
plt.title('AQI and 5-Day Average')
plt.xlabel('Date')
plt.ylabel('AQI')
plt.legend()
plt.show()

In [ ]:

#MANIPULATION

